In [1]:
import pandas as pd
from openai import OpenAI
from wardrobe import *
import time
import streamlit as st

client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

In [2]:
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written from the transcript of a youtube video. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully.
Please provide ONLY a numeric score based on the evaluation criteria and steps.
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """1. Read the summary and the source document carefully. 2. Compare the summary to the 
source document and identify the main points of the article. 3. Assess how well the summary covers the main points of 
the article, and how much irrelevant or redundant information it contains. 4. Assign a relevance score from 1 to 5."""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """1. Read the article carefully and identify the main topic and key points. 2. Read the 
summary and compare it to the article. Check if the summary covers the main topic and key points of the article, 
and if it presents them in a clear and logical order. 3. Assign a score for coherence on a scale of 1 to 5, 
where 1 is the lowest and 5 is the highest based on the Evaluation Criteria."""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """1. Read the article carefully and identify the main facts and details it presents. 2. 
Read the summary and compare it to the article. Check if the summary contains any factual errors that are not 
supported by the article. 3. Assign a score for consistency based on the Evaluation Criteria."""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, 
word choice, and sentence structure. 1: Poor. The summary has many errors that make it hard to understand or sound 
unnatural. 2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main 
points are still comprehensible. 3: Good. The summary has few or no errors and is easy to read and follow."""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""

In [3]:
model = "gpt-4"

def get_geval_score(
        criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    # print(prompt)

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        # temperature=1,
        # max_tokens=5,
        # top_p=1,
        # frequency_penalty=0,
        # presence_penalty=0,
    )
    return response.choices[0].message.content

In [4]:
evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

In [5]:
from openai import RateLimitError

summaries = {"Human Summary": ref_summary, "AI Summary 1": eval_summary_1, "AI Summary 2": eval_summary_2}
data = []

# Running the evaluation 3 times
for run_number in range(1, 4):  # Run numbers 1, 2, 3
    for eval_type, (criteria, steps) in evaluation_metrics.items():
        for summ_type, summary in summaries.items():
            max_attempts = 5
            for attempt in range(max_attempts):
                try:
                    result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
                    score_num = float(result.strip())
                    data.append({
                        "Run": run_number,
                        "Evaluation Type": eval_type,
                        "Summary Type": summ_type,
                        "Score": score_num
                    })
                    break
                except RateLimitError as e:
                    print(e)
                    if attempt < max_attempts - 1: # if not the last attempt
                        wait_time = (2 ** attempt) * 0.1 # exponential backoff
                        time.sleep(wait_time) # pause execution for wait_time
                    else: # if it was the last attempt
                        raise # re-raise the last exception

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-hZxykcrrUBPRLddRxyNM3Db7 on tokens_usage_based per min: Limit 10000, Used 8965, Requested 1169. Please try again in 804ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens_usage_based', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-hZxykcrrUBPRLddRxyNM3Db7 on tokens_usage_based per min: Limit 10000, Used 9033, Requested 1392. Please try again in 2.55s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens_usage_based', 'param': None, 'code': 'rate_limit_exceeded'}}
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-hZxykcrrUBPRLddRxyNM3Db7 on tokens_usage_based per min: Limit 10000, Used 9169, Requested 1203. Please try again in 2.232s. Visit https://platform.openai.com/account/rate-limits to learn m

In [6]:
data

[{'Run': 1,
  'Evaluation Type': 'Relevance',
  'Summary Type': 'Human Summary',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Relevance',
  'Summary Type': 'AI Summary 1',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Relevance',
  'Summary Type': 'AI Summary 2',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Coherence',
  'Summary Type': 'Human Summary',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Coherence',
  'Summary Type': 'AI Summary 1',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Coherence',
  'Summary Type': 'AI Summary 2',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Consistency',
  'Summary Type': 'Human Summary',
  'Score': 4.0},
 {'Run': 1,
  'Evaluation Type': 'Consistency',
  'Summary Type': 'AI Summary 1',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Consistency',
  'Summary Type': 'AI Summary 2',
  'Score': 5.0},
 {'Run': 1,
  'Evaluation Type': 'Fluency',
  'Summary Type': 'Human Summary',
  'Score': 3.0},
 {'Run': 1,
  'Evaluat

In [7]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Create a pivot table
pivot_df = df.pivot_table(
    index=["Run", "Evaluation Type"], 
    columns="Summary Type", 
    values="Score",
)
pivot_df.loc[('Total', ''), :] = pivot_df.sum()

In [8]:
import dataframe_image as dfi
# Define custom highlight function
def highlight_max_except_total(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    
    # Exclude 'Total' row for comparison
    data_no_total = data.drop(('Total', '')) if ('Total', '') in data.index else data
    
    # Identify the max value
    max_value = data_no_total.max()
    
    # Return styling for each element: highlight if it's a max value but not in 'Total' row
    return [attr if v == max_value and idx != ('Total', '') else '' for idx, v in data.items()]

# Apply the highlight function
styled_pivot_df = pivot_df.style.apply(highlight_max_except_total).format("{:.1f}")

# Display the styled pivot table
styled_pivot_df

In [9]:
dfi.export(styled_pivot_df, 'wardrobe.png', dpi=300)